In [1]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# from tensorflow.keras.metrics import categorical_accuracy
from sklearn.metrics import accuracy_score
from torchsummary import summary

GPU를 이용하기 위해 device를 환경에 맞게끔 초기화 해준다.

In [2]:
# CUDA를 이용한 GPU 가속
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(0))

# Mac M1에서 Pytorch 를 이용할 때 GPU 연산 가속을 위한 MPS
elif torch.torch.backends.mps.is_available():
    device = torch.device('mps:0')

    print("MPS enabled.")

# CPU 이용
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU: Tesla T4


Data를 학습시킬 때 필요한 여러 parameter 들을 config로 미리 정의

In [3]:
CFG = {
    'EPOCHS':30,
    'LEARNING_RATE':8e-3,
    'BATCH_SIZE':16,
    'SEED':42
}

모델의 성능을 재현하기 위해 seed값 고정

In [4]:
torch.manual_seed(CFG['SEED'])
np.random.seed(CFG['SEED'])
random.seed(CFG['SEED'])

In [5]:
# IRIS dataset 불러오기
iris = load_iris()
features = iris.data
labels = iris.target

load_iris()를 사용하여 불러온 것과 pandas를 활용해 새롭게 데이터 프레임을 구성

In [6]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [7]:
df['target'].value_counts()

0.0    50
1.0    50
2.0    50
Name: target, dtype: int64

Target class는 총 3개이며, 각 class는 50개의 데이터로 구성되어 있으므로, 전체 데이터는 balance 하다고 볼 수 있다.

train_test_split을 이용하여, train, validation, test dataset으로 분리한다.
먼저 train과 test를 8:2 의 비율로 나누고,
나눈 train을 다시 4:1의 비율로 train과 validation으로 나누어,
최종적으로 전체 dataset에 대해 train:validation:test = 6:2:2의 비율로 나누었다.

In [8]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=CFG['SEED'])
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=CFG['SEED'])

torch의 Dataset을 상속받아서 CustomDataset을 구현한다.
Dataset의 feature와 label을 input 값으로 입력받아 tensor로 변형하여 return

In [9]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, index):
        x = torch.tensor(self.features[index], dtype=torch.float32)
        y = torch.tensor(self.labels[index], dtype=torch.long)
        return x, y

    def __len__(self):
        return len(self.features)

In [10]:
train_dataset = CustomDataset(x_train, y_train)
val_dataset = CustomDataset(x_val, y_val)
test_dataset = CustomDataset(x_test, y_test)

torch의 DataLoader를 이용해서, 각각의 dataset을 이용해서 data loader를 정의

In [11]:
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'])
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'])

총 3개의 은닉충을 사용하여 딥러닝 모델을 구현하였다.
처음 input layer에서 4개의 feature 들을 512개의 차원으로 변형하고,
은닉층을 3개 거치면서, 절반의 크기의 dimension으로 변형되어 학습된다.
마지막으로 output layer에서 64개의 dimension을 target class 개수인 3개 변형되어 return 된다.
각각의 layer를 거칠 때 마다 relu 라는 활성화함수를 거쳐, 그 다음 layer로 넘겨준다.

In [12]:
class MyModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(MyModel,self).__init__()
        self.input_layer = nn.Linear(input_dim, 512)
        self.hidden_layer1 = nn.Linear(512, 256)
        self.hidden_layer2 = nn.Linear(256, 128)
        self.hidden_layer3 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()

    def forward(self,x):
        # 입력층
        out = self.input_layer(x)
        out = self.relu(out)

        # 은닉층
        out = self.hidden_layer1(out)
        out = self.relu(out)
        out = self.hidden_layer2(out)
        out = self.relu(out)
        out = self.hidden_layer3(out)
        out = self.relu(out)

        # 출력층
        out = self.output_layer(out)
        return out

model과 train, val loader 그리고 criterion(loss function) optimizer, epochs number 등을 파라미터로 받아 학습시키는 train함수를 정의한다.
epochs수 만큼 돌아가며 학습을 시키며, 그때마다 accuracy를 계산하여 가장 좋은 accuracy로 계속 갱신한다.

In [13]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_val_loss = float('inf')
    best_model_state_dict = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = []

        val_loss = []
        preds, trues = [], []

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # optimizer 행렬 0으로 초기화
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # loss를 계산하여 역전파
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        model.eval()

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                logit = model(inputs)
                loss = criterion(logit, labels)

                val_loss.append(loss.item())

                preds += logit.argmax(1).detach().cpu().numpy().tolist()
                trues += labels.detach().cpu().numpy().tolist()

            _val_loss = np.mean(val_loss)

        # _val_categorical_accuracy = np.mean(categorical_accuracy(trues, preds))
        _val_accuracy = accuracy_score(trues, preds)
        _train_loss = np.mean(train_loss)

        print(f"Epoch {epoch+1}: "
              f"Train Loss: {_train_loss:.4f}, "
              f"Val Loss: {_val_loss:.4f}, "
              f"Val Accuracy: {_val_accuracy:.4f}")

        # loss값이 이전까지 loss값의 최소값보다 작을 경우, 가장 좋은 성능의 model로 갱신한다.
        if _val_loss < best_val_loss:
            best_val_loss = _val_loss
            best_model_state_dict = model.state_dict()

    # best model을 불러와서 return
    model.load_state_dict(best_model_state_dict)
    return model

In [14]:
def test(model, test_loader, criterion):
    test_loss = 0.0
    accuracy = 0.0
    num_samples = len(test_loader.dataset)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            accuracy += (predicted == labels).sum().item()

    avg_accuracy = accuracy / num_samples * 100

    print(f"Test Accuracy: {avg_accuracy:.2f}%")

features 의 개수가 4개고, target class의 개수가 3개이므로, 입력층의 dimension을 4, 출력층의 dimension을 3 으로 지정하여 model을 정의해준다.
정의한 model을 GPU로 연산하기 위해, model.to(device)를 해준다.

정의한 model의 구조를 보기위해, torchsummary를 이용해서 model의 아키텍처를 출력한다.
batch size가 16이고, input dimension이 4 이기 때문에, (16, 4)를 input shape으로 각 layer를 통과하며, 각 layer를 통과할 때 마다 output이 어떻게 되는지 보았다.

In [15]:
model = MyModel(4, 3)
model.to(device)

summary(model, (16, 4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 16, 512]           2,560
              ReLU-2              [-1, 16, 512]               0
            Linear-3              [-1, 16, 256]         131,328
              ReLU-4              [-1, 16, 256]               0
            Linear-5              [-1, 16, 128]          32,896
              ReLU-6              [-1, 16, 128]               0
            Linear-7               [-1, 16, 64]           8,256
              ReLU-8               [-1, 16, 64]               0
            Linear-9                [-1, 16, 3]             195
Total params: 175,235
Trainable params: 175,235
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.23
Params size (MB): 0.67
Estimated Total Size (MB): 0.90
-------------------------------------------

loss function은 CrossEntropyLoss를 사용하고,
optimizer는 SGD를 사용하였다. 이때 momentum의 값은 0.9로 한다.
learning_rate는 8e-3, EPOCHS수는 30으로 정하여 train 시킨다.

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=CFG['LEARNING_RATE'], momentum=0.9)

train(model, train_loader, val_loader, criterion, optimizer, CFG['EPOCHS'])

Epoch 1: Train Loss: 1.1050, Val Loss: 1.0799, Val Accuracy: 0.3000
Epoch 2: Train Loss: 1.0577, Val Loss: 1.0442, Val Accuracy: 0.5667
Epoch 3: Train Loss: 0.9969, Val Loss: 0.9850, Val Accuracy: 0.5667
Epoch 4: Train Loss: 0.8753, Val Loss: 0.8396, Val Accuracy: 0.5667
Epoch 5: Train Loss: 0.6695, Val Loss: 0.6245, Val Accuracy: 0.5667
Epoch 6: Train Loss: 0.4684, Val Loss: 0.5145, Val Accuracy: 0.6000
Epoch 7: Train Loss: 0.3897, Val Loss: 0.4185, Val Accuracy: 0.7333
Epoch 8: Train Loss: 0.3856, Val Loss: 0.4170, Val Accuracy: 0.7333
Epoch 9: Train Loss: 0.2263, Val Loss: 0.3228, Val Accuracy: 0.8667
Epoch 10: Train Loss: 0.2859, Val Loss: 0.3303, Val Accuracy: 0.8333
Epoch 11: Train Loss: 0.1905, Val Loss: 0.5516, Val Accuracy: 0.7667
Epoch 12: Train Loss: 0.2214, Val Loss: 0.1724, Val Accuracy: 0.9000
Epoch 13: Train Loss: 0.1070, Val Loss: 0.2525, Val Accuracy: 0.9000
Epoch 14: Train Loss: 0.1777, Val Loss: 0.5264, Val Accuracy: 0.7667
Epoch 15: Train Loss: 0.5995, Val Loss: 0.4

MyModel(
  (input_layer): Linear(in_features=4, out_features=512, bias=True)
  (hidden_layer1): Linear(in_features=512, out_features=256, bias=True)
  (hidden_layer2): Linear(in_features=256, out_features=128, bias=True)
  (hidden_layer3): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=3, bias=True)
  (relu): ReLU()
)

학습을 시켜, 각 epochs마다 loss값과 accuracy를 계산하여 계속 학습시키니, 점차 loss값이 줄어들고, accuracy값은 상승하는 것을 볼 수 있다.
train loss만 줄어들고, validation loss가 먼저 높은 값에서 수렴한다면, over fitting이 된 것이라 해석할 수 있겠으나,
train loss와 validation loss 가 함께 줄어들며 학습되고 있으므로, model이 잘 학습되었음을 알 수 있다.
처음에 learning rate를 1e-2로 하여 학습시켰으나, loss 값이 안정적으로 수렴하는 모습을 보이지않아, 차츰 줄여가며 학습시켜보았다.
1e-3으로 하니 너무 일찍 validation loss가 수렴하여, 원하는 성능이 나오지 않았다.
최종적으로 8e-3을 learning rate로 선정하였으며, 이 learning rate에서 model의 학습이 가장 안정적으로 진행되는 것을 확인하였다.

In [17]:
test(model, test_loader, criterion)

Test Accuracy: 96.67%


학습한 model을 test dataset에 대하여 검증해보니, 96.67%의 accuracy를 달성할 수 있었다.